<a href="https://colab.research.google.com/github/CamiloVga/Prueba-Tecnica_Bid/blob/main/Ejercicio_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 2

En este reto se aborda la implementación de un sistema RAG (Retrieval Augmented Generation) diseñado para potenciar el análisis de documentos fiscales del BID.

La arquitectura se construye sobre langchain, que sirve como columna vertebral del sistema, e incorpora dos componentes clave: el modelo de lenguaje Llama-2-7b-chat-hf de HuggingFace, responsable del procesamiento y generación de respuestas, y los embeddings multilingual-e5-large, que permiten una interpretación precisa de documentos técnicos en diversos idiomas. Esta combinación de tecnologías permite al sistema comprender y procesar eficientemente la documentación técnica fiscal, independientemente del idioma en que esté escrita.

La estretegia para el ejercicio se explica en detalle en este Script. Pero pueden ver el **producto final** en esta app diseñada para brindar una buena **experiencia de usuario:** https://huggingface.co/spaces/CamiloVega/Fislac_Bot


**Recomendaciones de uso del script**

1. El script está diseñado para ejecutarse en Google Colab, lo que elimina la necesidad de configuraciones locales complejas.

2. Requisitos importantes:
   - Se necesita una cuenta en Hugging Face
   - Requerirá un token de acceso de Hugging Face y guardarlo como un secreto en Google Colab
   - Se recomienda tener acceso a modelos de Llama 2 en Hugging Face

3. Archivos de prueba:
   - El sistema espera documentos PDF sobre FISLAC y documentos técnicos del BID
   - Los archivos deben subirse cuando aparezca el widget de carga en Colab

4. Recursos computacionales:
   - Se requiere un runtime con GPU (T4 o mejor)
   - Memoria RAM recomendada: 12GB o superior
   - El modelo se carga en 8-bit para optimizar memoria



# 1. Configuraciones iniciales
El sistema RAG se construye sobre una base de componentes seleccionados para el procesamiento eficiente de documentación fiscal. Como núcleo del sistema, PyPDF2 y pdfplumber manejan la extracción de texto de documentos PDF, mientras langchain-community proporciona el framework para construir la aplicación contextual.

La elección de estas herramientas cobra relevancia al procesar documentos técnicos del BID: FAISS gestiona la indexación y búsqueda vectorial, permitiendo recuperar información precisa de documentos extensos. Para el procesamiento multilingüe, transformers y sentence-transformers de Hugging Face forman la columna vertebral del sistema, respaldados por un sistema de logging para monitoreo y typing para integridad del código.

El modelo multilingual-e5-large se utiliza específicamente para generar los embeddings vectoriales de los textos, mejorando la representación semántica de conceptos fiscales en múltiples idiomas. Tras varias pruebas se encontró que este modelo (e5-large) es superior frente alternativas como flan-t5-base y paraphrase-multilingual-MiniLM-L12-v2. Este componente es crucial para la calidad de las búsquedas y recuperación de información relevante

In [ ]:
# Instalación de dependencias necesarias
!pip install -q pypdf
!pip install -q PyPDF2 pdfplumber
!pip install -q langchain-community
!pip install -q faiss-cpu
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q torch
!pip install -q gradio
!pip install -q accelerate
!pip install -q kitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB

In [ ]:
# Módulos básicos necesarios para el funcionamiento general de la aplicación
import os
import logging
from typing import List, Dict, Optional

# Componentes para el procesamiento de PDFs
# Utilizaremos estas herramientas para extraer y manipular el contenido de documentos PDF,
# que servirá como fuente de información para nuestro sistema
from pypdf import PdfReader
from langchain_community.document_loaders import PyPDFLoader

# Núcleo del procesamiento de texto y gestión de embeddings
# Este bloque nos permite dividir textos largos en fragmentos más manejables,
# convertirlos en vectores y almacenarlos para búsquedas eficientes
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Componentes para la generación de respuestas y gestión de prompts
# Estas herramientas nos permiten crear un sistema de preguntas y respuestas
# basado en la información procesada de los PDFs
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Herramientas de Transformers para el procesamiento del lenguaje natural
# Aquí importamos los componentes necesarios para trabajar con modelos de lenguaje
# y crear pipelines de procesamiento automático
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline
)

# Integraciones específicas para el entorno de trabajo
# Estas importaciones nos permiten trabajar con modelos de HuggingFace en LangChain
# y manejar archivos en el entorno de Google Colab
from langchain.llms import HuggingFacePipeline
from google.colab import files

# Configuración del sistema de logs para monitorear el funcionamiento de la aplicación
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## 1.2 Base de conocimiento
En esta parte se prepara el directorio de la base de conocimiento y se abre el widget para subir los documentos en PDF sobre los cuales se quiera aplicar el sistema RAG

In [ ]:
# Limpiar y crear directorio para la base de conocimiento
!rm -rf knowledge_base
!mkdir knowledge_base

print("Por favor, sube tus archivos PDF cuando aparezca el botón...")
uploaded = files.upload()

# Guardar los archivos en el directorio knowledge_base
for filename, content in uploaded.items():
    if filename.endswith('.pdf'):
        path = f'knowledge_base/{filename}'
        with open(path, 'wb') as f:
            f.write(content)
        print(f"Archivo {filename} guardado en knowledge_base/")

# Verificar archivos subidos
!ls knowledge_base

Por favor, sube tus archivos PDF cuando aparezca el botón...


Saving 22. Valencia, Díaz, Parra (2022) - Assessing macro-fiscal risk for Latin American and Caribbean countries.pdf to 22. Valencia, Díaz, Parra (2022) - Assessing macro-fiscal risk for Latin American and Caribbean countries.pdf
Saving fislac_kb.pdf to fislac_kb.pdf
Archivo 22. Valencia, Díaz, Parra (2022) - Assessing macro-fiscal risk for Latin American and Caribbean countries.pdf guardado en knowledge_base/
Archivo fislac_kb.pdf guardado en knowledge_base/
'22. Valencia, Díaz, Parra (2022) - Assessing macro-fiscal risk for Latin American and Caribbean countries.pdf'
 fislac_kb.pdf


# 2. Estructuración del sistema RAG

El sistema RAG procesa y responde consultas sobre documentos fiscales del BID mediante tres componentes principales.

1. DocumentLoader maneja la carga inicial de PDFs, clasificando automáticamente cada documento como "técnico" o "preguntas y respuestas" (Q&A), y agregando metadatos útiles como el idioma y tipo de contenido.

2. TextProcessor divide los documentos en fragmentos de texto (chunks) usando una técnica de "ventana deslizante", similar a cómo leeríamos un libro largo usando un marcador que se mueve gradualmente:



*   Para documentos técnico se crean fragmentos de 800 tokens, donde cada nuevo fragmento retiene 200 tokens del fragmento anterior para mantener el contexto. Es como si al leer un libro técnico, cada vez que avanzamos una página nueva, releemos el último párrafo de la página anterior para no perder el hilo.
*   Para documentos Q&A (como el archivo Fislac) se usan fragmentos más cortos de 500 tokens, con una superposición de 100 tokens, ya que estos documentos suelen tener información más concisa y directa. Estos fragmentos procesados se convierten en vectores numéricos (embeddings) usando el modelo multilingual-e5-large, que FAISS organiza eficientemente para búsquedas rápidas.

3. El sistema culmina con un módulo de recuperación que selecciona los 6 fragmentos más relevantes para cada consulta, alimentándolos al modelo Llama-2-7b-chat-hf junto con un prompt template en español. Esta configuración permite mantener el contexto técnico-fiscal mientras adapta las respuestas según el tipo de documento y consulta.


In [ ]:
#Clases del Sistema

class DocumentLoader:
    """Clase para gestionar la carga de documentos PDF."""

    @staticmethod
    def load_pdfs(directory_path: str) -> List:
        documents = []
        pdf_files = [f for f in os.listdir(directory_path) if f.endswith('.pdf')]

        for pdf_file in pdf_files:
            pdf_path = os.path.join(directory_path, pdf_file)
            try:
                loader = PyPDFLoader(pdf_path)
                pdf_documents = loader.load()

                # Metadatos mejorados para contextualización
                for doc in pdf_documents:
                    doc.metadata.update({
                        'title': pdf_file,
                        'type': 'technical' if 'Valencia' in pdf_file else 'qa',
                        'language': 'es',
                        'page': doc.metadata.get('page', 0)
                    })
                    documents.append(doc)

                logger.info(f"Documento {pdf_file} cargado exitosamente")
            except Exception as e:
                logger.error(f"Error al cargar {pdf_file}: {str(e)}")

        return documents

class TextProcessor:
    """Clase para procesar y dividir el texto en chunks."""

    def __init__(self):
        self.technical_splitter = RecursiveCharacterTextSplitter(
            chunk_size=800,
            chunk_overlap=200,
            separators=["\n\n", "\n", ". ", " ", ""],
            length_function=len,
            is_separator_regex=False
        )

        self.qa_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=100,
            separators=["\n\n", "\n", ". ", " ", ""],
            length_function=len,
            is_separator_regex=False
        )

    def process_documents(self, documents: List) -> List:
        """Procesa los documentos con configuración específica según tipo."""
        if not documents:
            logger.warning("No hay documentos para procesar")
            return []

        processed_chunks = []
        for doc in documents:
            # Seleccionar el splitter según el tipo de documento
            splitter = self.technical_splitter if doc.metadata['type'] == 'technical' else self.qa_splitter

            chunks = splitter.split_documents([doc])
            processed_chunks.extend(chunks)

        logger.info(f"Documentos procesados en {len(processed_chunks)} fragmentos")
        return processed_chunks

class RAGSystem:
    """Sistema principal de RAG con configuración optimizada."""

    def __init__(self, model_name: str = "meta-llama/Llama-2-7b-chat-hf"):
        self.model_name = model_name
        self.embeddings = None
        self.vector_store = None
        self.qa_chain = None
        self.tokenizer = None
        self.model = None

    def initialize_embeddings(self):
        """Inicializa embeddings con modelo multilingüe optimizado."""
        try:
            self.embeddings = HuggingFaceEmbeddings(
                model_name="intfloat/multilingual-e5-large",
                model_kwargs={'device': 'cuda'},
                encode_kwargs={'normalize_embeddings': True}
            )
            logger.info("Modelo de embeddings inicializado exitosamente")
        except Exception as e:
            logger.error(f"Error al inicializar embeddings: {str(e)}")
            raise

    def create_vector_store(self, processed_chunks: List):
        """Crea almacén de vectores con configuración mejorada."""
        if not self.embeddings:
            raise ValueError("Embeddings no inicializados. Ejecute initialize_embeddings primero.")

        try:
            self.vector_store = FAISS.from_documents(
                processed_chunks,
                self.embeddings
            )
            logger.info("Almacén de vectores creado exitosamente")
        except Exception as e:
            logger.error(f"Error al crear vector store: {str(e)}")
            raise

    def setup_rag_pipeline(self):
        """Configura pipeline RAG con prompt mejorado."""
        if not self.vector_store:
            raise ValueError("Vector store no inicializado")

        try:
            # Configuración del modelo
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                load_in_8bit=True,
                device_map="auto",
                trust_remote_code=True
            )

            # Pipeline de generación optimizado
            pipe = pipeline(
                "text-generation",
                model=self.model,
                tokenizer=self.tokenizer,
                max_length=2048,
                temperature=0.1,
                top_p=0.95,
                repetition_penalty=1.15,
                truncation=True
            )

            llm = HuggingFacePipeline(pipeline=pipe)

            # Prompt template simplificado
            prompt_template = """
            Contexto: {context}

            Basándote en el contexto anterior, responde a la siguiente pregunta de manera clara y concisa en español.
            Si la información no está en el contexto, indícalo explícitamente.

            Pregunta: {question}
            """

            PROMPT = PromptTemplate(
                template=prompt_template,
                input_variables=["context", "question"]
            )

            # Configuración de la cadena QA
            self.qa_chain = RetrievalQA.from_chain_type(
                llm=llm,
                chain_type="stuff",
                retriever=self.vector_store.as_retriever(
                    search_kwargs={"k": 6, "score_threshold": 0.5}
                ),
                return_source_documents=True,
                chain_type_kwargs={"prompt": PROMPT}
            )

            logger.info("Pipeline RAG configurado exitosamente")
        except Exception as e:
            logger.error(f"Error al configurar pipeline RAG: {str(e)}")
            raise

    def answer_question(self, question: str) -> Dict:
        """Procesa una pregunta y genera una respuesta basada en el contexto recuperado."""
        if not self.qa_chain:
            raise ValueError("Pipeline RAG no configurado. Ejecute setup_rag_pipeline primero.")

        try:
            # Procesar la pregunta
            result = self.qa_chain({"query": question})

            # Estructurar la respuesta
            response = {
                'answer': result['result'],
                'sources': []
            }

            # Procesar documentos fuente
            for doc in result['source_documents']:
                source = {
                    'title': doc.metadata.get('title', 'Unknown'),
                    'content': doc.page_content[:200] + "..." if len(doc.page_content) > 200 else doc.page_content,
                    'metadata': doc.metadata,
                    'score': doc.metadata.get('score', 0.0)
                }
                response['sources'].append(source)

            return response

        except Exception as e:
            logger.error(f"Error al procesar la pregunta: {str(e)}")
            raise

def test_rag_system(question: str):
    """Función que muestra solo pregunta, respuesta y fuentes."""
    try:
        response = rag_system.answer_question(question)

        # Extraer solo la respuesta final después de "Respuesta:"
        answer = response['answer']
        if "Respuesta:" in answer:
            answer = answer.split("Respuesta:")[-1].strip()
        elif "Respuesta (en español):" in answer:
            answer = answer.split("Respuesta (en español):")[-1].strip()

        # Eliminar todo el texto antes de la respuesta real
        if "Basándote en el contexto anterior" in answer:
            answer = answer.split("Basándote en el contexto anterior")[0].strip()

        # Imprimir solo la información esencial
        print(f"Pregunta: {question}")
        print(f"Respuesta: {answer}\n")
        print("Fuentes utilizadas:")

        # Imprimir solo las fuentes relevantes
        for source in response['sources'][:3]:  # Limitamos a 3 fuentes
            doc_title = source['title']
            doc_type = source['metadata']['type']
            excerpt = source['content'][:100] + "..." if len(source['content']) > 100 else source['content']
            print(f"- {doc_title} ({doc_type})")
            print(f"  {excerpt}\n")

        print("-" * 80 + "\n")  # Separador visual

    except Exception as e:
        print(f"Error al procesar la pregunta: {str(e)}")

# 2.2 Estructuración del sistema RAG

En esta sección implementamos la secuencia de inicialización y configuración del pipeline RAG, que se desarrolla en tres fases:

1. La fase inicial verifica la presencia de documentos PDF en knowledge_base mediante una validación automatizada.

2. DocumentLoader procesa los PDFs enriqueciendo los metadatos con información esencial como tipo de documento (técnico o Q&A) e idioma. TextProcessor segmenta los documentos según su naturaleza: fragmentos extensos para documentos técnicos y compactos para Q&A.

2. El sistema RAG se configura secuencialmente: configura embeddings con multilingual-e5-large optimizado para GPU, construye el vector store con FAISS, y finaliza con el pipeline Llama-2-7b-chat-hf usando cuantización de 8-bits para optimizar memoria. El sistema monitorea cada etapa, reportando la cantidad de documentos procesados, chunks generados y el estado de los componentes

In [ ]:
# Inicialización y configuración del sistema
print("Verificando documentos en el directorio...")
pdf_files = [f for f in os.listdir("knowledge_base") if f.endswith('.pdf')]
if not pdf_files:
    raise ValueError("No hay archivos PDF en knowledge_base. Por favor, sube algunos archivos primero.")

print(f"Encontrados {len(pdf_files)} archivos PDF")

# Cargar documentos
loader = DocumentLoader()
documents = loader.load_pdfs("knowledge_base")

if not documents:
    raise ValueError("No se pudieron cargar los documentos")

print(f"Se cargaron {len(documents)} documentos")

# Procesar documentos
processor = TextProcessor()
processed_chunks = processor.process_documents(documents)

print(f"Se crearon {len(processed_chunks)} chunks")

# Configurar sistema RAG
rag_system = RAGSystem()
rag_system.initialize_embeddings()
rag_system.create_vector_store(processed_chunks)
rag_system.setup_rag_pipeline()

print("Sistema RAG creado exitosamente")

Verificando documentos en el directorio...
Encontrados 2 archivos PDF
Se cargaron 47 documentos
Se crearon 216 chunks


<ipython-input-4-fe8c5e958fba>:84: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Sistema RAG creado exitosamente


<ipython-input-4-fe8c5e958fba>:136: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


# 3. Pruebas del sistema
La fase de pruebas evalúa la capacidad del sistema RAG para procesar y responder consultas sobre documentación fiscal. La función test_rag_system centraliza esta evaluación, procesando las consultas y proporcionando transparencia sobre las fuentes utilizadas.

Para garantizar una validación efectiva, el sistema prueba tres escenarios clave:
1. Preguntas sobre el contenido técnico del paper de Valencia et al., enfocándose en variables macroeconómicas y análisis comparativos entre países
2. Consultas específicas sobre la plataforma FISLAC, sus módulos y funcionalidades
3. Preguntas que requieren la integración y síntesis de información de ambas fuentes

El sistema procesa cada consulta mostrando la respuesta generada junto con los metadatos de las fuentes utilizadas.

In [ ]:
def test_rag_system(question: str):
    """Función mejorada para probar el sistema RAG."""
    print(f"\nPregunta: {question}")

    try:
        response = rag_system.answer_question(question)

        print("\nRespuesta:", response['answer'])
        print("\nFuentes utilizadas:")
        for source in response['sources']:
            print(f"- Documento: {source['title']} (Tipo: {source['metadata']['type']})")
            print(f"  Extracto: {source['content']}")
            print(f"  Confianza: {source['score']:.2f}")
        print("\n" + "="*50)
    except Exception as e:
        print(f"Error al procesar la pregunta: {str(e)}")

# Lista de preguntas de prueba
test_questions = [
    "¿Qué es FISLAC?",
    "¿Cuáles son los módulos principales de FISLAC?",
    "¿Qué variables macroeconómicas son más relevantes para economías avanzadas según el estudio?",
    "¿Cómo se compara el riesgo fiscal entre países emergentes y avanzados?",
    "¿Cómo se relaciona el análisis de riesgo fiscal del paper con las herramientas de FISLAC?"
]

print("Ejecutando pruebas con preguntas diversas...")
for question in test_questions:
    test_rag_system(question)

<ipython-input-4-fe8c5e958fba>:176: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = self.qa_chain({"query": question})


Ejecutando pruebas con preguntas diversas...

Pregunta: ¿Qué es FISLAC?

Respuesta: 
            Contexto: What is FISLAC? 
FISLAC (Fiscal Sustainability for Latin American and the Caribbean countries) is an 
ecosystem developed by the Inter-American Development Bank to help governments of 
Latin American and Caribbean countries (LAC) strengthen macro -fiscal policy and 
decision-making. 
FISLAC is comprised of two primary elements: the models and the website. The models 
serve as the core of the tools displayed on the website. Upon exploration, within the

serve as the core of the tools displayed on the website. Upon exploration, within the 
website you will discover a variety of dashboards and tools designed to facilitate a 
comprehensive understanding of the macro-fiscal dynamics of LAC countries. 
Why FISLAC? 
FISLAC is an agile and accessible platform that provides an overview of the macro-fiscal 
risks faced by countries in the LAC region. 
How can I access FISLAC? 
To access the

#3.1 Inferencia por medio de preguntas interactivas

Esta sección implementa una interfaz conversacional que permite a los usuarios interactuar con el sistema RAG en tiempo real, similar a un chatbot especializado en documentación fiscal del BID. Para facilitar esta interacción, se creó la función ask_custom_question(), que establece una sesión de diálogo continua entre el usuario y el sistema.

In [ ]:
#Función interactiva
def ask_custom_question():
    while True:
        question = input("\nEscribe tu pregunta (o 'q' para salir): ").strip()
        if question.lower() == 'q':
            break
        test_rag_system(question)

print("\nAhora puedes hacer tus propias preguntas:")
ask_custom_question()


Ahora puedes hacer tus propias preguntas:

Escribe tu pregunta (o 'q' para salir): q


## 3.2 Análisis resultados
El análisis de las respuestas del sistema RAG demuestra su efectividad para procesar e integrar información de múltiples documentos técnicos, con capacidades destacadas en traducción inglés-español y mantenimiento del contexto técnico-fiscal.

La siguiente fase de desarrollo expandiría las capacidades mediante una arquitectura avanzada: implementación de PostgreSQL para gestión eficiente de embeddings y metadatos, uso del framework Groq para optimizar la inferencia, y despliegue en contenedores para escalabilidad. La integración de estas tecnologías, junto con la optimización de la segmentación de documentos y un sistema de puntuación avanzado para las fuentes, permitiría respuestas más precisas y tiempos de inferencia reducidos. Este enfoque profesional consolidaría al sistema como una herramienta robusta para análisis fiscal en entornos de producción.